In [19]:
import pandas as pd
import numpy as np

In [3]:
# Read all-word data (.csv)
d_allwork_df = pd.read_csv(r'Data/d_all_work.csv')
s_allwork_df = pd.read_csv(r'Data/s_all_work.csv')
d_event_df = pd.read_csv(r'Data/d_events.csv')
s_event_df = pd.read_csv(r'Data/s_events.csv')

#d_allwork_df.head(1)
#s_allwork_df.head(1)
d_event_df.head(1)
#s_event_df.head(1)

,#,Workgroup ID,WISE ID 1,Student Name 1,WISE ID 2,Student Name 2,WISE ID 3,Student Name 3,Class Period,Project ID,...,Step Title,Component Type,Component Prompt,Group Event Counter,Context,Category,Event,Event Data,Response,Unnamed: 29
0,1,6740,4757,lilingqi d,NaN,NaN,NaN,NaN,1,891,...,1.1: 介绍,NaN,NaN,1,VLE,Navigation,nodeEntered,"{""serverSaveTime"":1575343781000,""periodId"":761...",,NaN


In [4]:
def get_names(df):
    df = df['Student Name 1']
    n_list = []
    for name in df:
        if name not in n_list:
            n_list.append(name)
    return n_list

s_names = get_names(s_allwork_df)
d_names = get_names(d_allwork_df)

# print(s_names[:10])
# print(d_names[:10])

# Step 1.6
1. Time spends on page 1.6 on the first day; The rest of time spend on page 1.6

In [99]:
# 1.6.1

from datetime import datetime
FMT = '%d,%H:%M:%S'

"""
def step161(df_161):
    duration_list = []
    for name in s_names:
        df = df_161.loc[df_161['Student Name 1'] == name][['Server Timestamp', 'Teacher Comment']]
        # print(df)
        time_list = []
        if len(df) < 2:
            print('<', name, '> has less than 2 entries in [1.6]') 
            continue
        else:
            idx_list = df.index
            time_list = df['Server Timestamp'][[idx_list[0], idx_list[-1]]].tolist()
            duration = {}
            day_time = []
            for __, entry in df.iterrows():
                if str(entry[1]) != 'nan':
                    time_list.append(str(entry[0]))
                    break        
            for time_str in time_list:
                time_slice = time_str.split(' ')
                #print(time_slice)
                day = time_slice[2]
                time = time_slice[-2]
                day_time.append(str(day) + ',' + str(time))
            duration['Before'] = (datetime.strptime(day_time[-1], FMT) - datetime.strptime(day_time[0], FMT)).seconds
            if(len(day_time) > 2):
                after = (datetime.strptime(day_time[1], FMT) - datetime.strptime(day_time[-1], FMT)).seconds
                duration['After'] = after if after > 0 else -1
            else:
                duration['After'] = -1
            duration_list.append((name, duration))
    return duration_list
"""

def step_161(df_161, names):
    duration_df = pd.DataFrame()
    idx = 0
    for name in names:
        df = df_161.loc[df_161['Student Name 1'] == name][['Server Timestamp', 'Event']]
        # print(df)
        time_list = []
        duration = {'Student Name 1': name}
        counter = 0 # 0 means need enter, 1 means need exit
        if len(df) < 2:
            print('<', name, '> has less than 2 entries in [1.6]') 
            continue
        else:
            for __, entry in df.iterrows():
                time_slice = str(entry[0]).split(' ')
                day = time_slice[2]
                time = time_slice[-1][2:]
                day_time = str(day) + ',' + str(time)
                if counter == 0 and str(entry[1]) == 'nodeEntered':
                    time_list.append(day_time)
                    counter = 1
                elif counter == 1 and str(entry[1]) == 'nodeExited':
                    time_list.append(day_time)
                    counter = 0
                else:
                    continue
            if len(time_list) % 2 != 0:
                time_list = time_list[0:-1]
            duration['Time1.6_Before'] = (datetime.strptime(time_list[1], FMT) - datetime.strptime(time_list[0], FMT)).seconds
            if len(time_list) == 2:
                duration['Time1.6_After'] = -1
            else:
                duration['Time1.6_After'] = 0
                for i in range(2, len(time_list), 2):
                    duration['Time1.6_After'] += (datetime.strptime(time_list[i+1], FMT) - datetime.strptime(time_list[i], FMT)).seconds
        duration_df = pd.concat([duration_df, pd.DataFrame(duration, index=[idx])], ignore_index=True)
        idx += 1
    return duration_df

def rw_161(df, names, output):
    df_161 = df.loc[df['Step Title'].str.contains('1.6', na=False)]
    df_161 = step_161(df_161, names)
    #print(step161(df_161))
    for idx, entry in df_161.iterrows():
        if entry[1] > 10000:
            df_161.loc[idx, 'Time1.6_Before'] = -2
    path_name = 'Result/' + output
    df_161.to_csv(path_name)
    print('Write to ' + path_name)
    
rw_161(s_event_df, s_names, 's_161.csv')
rw_161(d_event_df, d_names, 'd_161.csv')


Write to Result/s_161.csv
Write to Result/d_161.csv


# Step 1.6
2. Number of revisiting before first feedback (after arriving on page 1.6, go back to previous pages)
Number of revisiting after first feedback

In [23]:
def step_162(df_162, names):
    for name in names:
        df = df_162.loc[df_162['Student Name 1'] == name]
        step_dict = {}
        for idx, row in df.iterrows():
            #print(row['Server Timestamp'])
            day = row['Server Timestamp'].split(' ')[2]
            if day not in step_dict:
                step_dict[day] = [row['Step Title']]
            else:
                step_dict[day].append(row['Step Title'])
        print(step_dict)



def rw_162(df, names, output):
    df_162 = df[['Student Name 1', 'Server Timestamp', 'Step Title']]
    df_162 = step_162(df_162, names)

    # path_name = 'Result/' + output
    # df_162.to_csv(path_name)
    # print('Write to ' + path_name)
    
rw_162(s_event_df, s_names, 's_162.csv')
#rw_162(d_event_df, d_names, 'd_162.csv')

Tue Dec 03 2019 下午4:16:48
Tue Dec 03 2019 下午4:16:48
Tue Dec 03 2019 下午4:17:16
Tue Dec 03 2019 下午4:17:16
Tue Dec 03 2019 下午4:17:42
Tue Dec 03 2019 下午4:17:42
Tue Dec 03 2019 下午4:18:37
Tue Dec 03 2019 下午4:18:37
Tue Dec 03 2019 下午4:24:54
Tue Dec 03 2019 下午4:25:45
Tue Dec 03 2019 下午4:25:59
Tue Dec 03 2019 下午4:26:26
Tue Dec 03 2019 下午4:26:41
Tue Dec 03 2019 下午4:26:41
Tue Dec 03 2019 下午4:27:43
Tue Dec 03 2019 下午4:27:56
Tue Dec 03 2019 下午4:28:16
Tue Dec 03 2019 下午4:28:29
Tue Dec 03 2019 下午4:28:29
Tue Dec 03 2019 下午4:36:11
Tue Dec 03 2019 下午4:36:11
Thu Dec 05 2019 上午10:24:38
Thu Dec 05 2019 上午10:24:38
Thu Dec 05 2019 上午10:28:24
Thu Dec 05 2019 上午10:28:24
Thu Dec 05 2019 上午10:40:40
Thu Dec 05 2019 上午10:40:40
Thu Dec 05 2019 上午10:44:28
Thu Dec 05 2019 上午10:44:28
Thu Dec 05 2019 上午10:46:43
Thu Dec 05 2019 上午10:46:43
Thu Dec 05 2019 上午10:46:49
Thu Dec 05 2019 上午10:47:07
Thu Dec 05 2019 上午10:47:11
Thu Dec 05 2019 上午10:47:40
Thu Dec 05 2019 上午10:47:40
Thu Dec 05 2019 上午10:48:19
Thu Dec 05 2019 上午10:4